# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [6]:
vocab_to_int

{'the': 0,
 'of': 1,
 'and': 2,
 'one': 3,
 'in': 4,
 'a': 5,
 'to': 6,
 'zero': 7,
 'nine': 8,
 'two': 9,
 'is': 10,
 'as': 11,
 'eight': 12,
 'for': 13,
 's': 14,
 'five': 15,
 'three': 16,
 'was': 17,
 'by': 18,
 'that': 19,
 'four': 20,
 'six': 21,
 'seven': 22,
 'with': 23,
 'on': 24,
 'are': 25,
 'it': 26,
 'from': 27,
 'or': 28,
 'his': 29,
 'an': 30,
 'be': 31,
 'this': 32,
 'which': 33,
 'at': 34,
 'he': 35,
 'also': 36,
 'not': 37,
 'have': 38,
 'were': 39,
 'has': 40,
 'but': 41,
 'other': 42,
 'their': 43,
 'its': 44,
 'first': 45,
 'they': 46,
 'some': 47,
 'had': 48,
 'all': 49,
 'more': 50,
 'most': 51,
 'can': 52,
 'been': 53,
 'such': 54,
 'many': 55,
 'who': 56,
 'new': 57,
 'used': 58,
 'there': 59,
 'after': 60,
 'when': 61,
 'into': 62,
 'american': 63,
 'time': 64,
 'these': 65,
 'only': 66,
 'see': 67,
 'may': 68,
 'than': 69,
 'world': 70,
 'i': 71,
 'b': 72,
 'would': 73,
 'd': 74,
 'no': 75,
 'however': 76,
 'between': 77,
 'about': 78,
 'over': 79,
 'years': 

In [7]:
int_to_vocab

{0: 'the',
 1: 'of',
 2: 'and',
 3: 'one',
 4: 'in',
 5: 'a',
 6: 'to',
 7: 'zero',
 8: 'nine',
 9: 'two',
 10: 'is',
 11: 'as',
 12: 'eight',
 13: 'for',
 14: 's',
 15: 'five',
 16: 'three',
 17: 'was',
 18: 'by',
 19: 'that',
 20: 'four',
 21: 'six',
 22: 'seven',
 23: 'with',
 24: 'on',
 25: 'are',
 26: 'it',
 27: 'from',
 28: 'or',
 29: 'his',
 30: 'an',
 31: 'be',
 32: 'this',
 33: 'which',
 34: 'at',
 35: 'he',
 36: 'also',
 37: 'not',
 38: 'have',
 39: 'were',
 40: 'has',
 41: 'but',
 42: 'other',
 43: 'their',
 44: 'its',
 45: 'first',
 46: 'they',
 47: 'some',
 48: 'had',
 49: 'all',
 50: 'more',
 51: 'most',
 52: 'can',
 53: 'been',
 54: 'such',
 55: 'many',
 56: 'who',
 57: 'new',
 58: 'used',
 59: 'there',
 60: 'after',
 61: 'when',
 62: 'into',
 63: 'american',
 64: 'time',
 65: 'these',
 66: 'only',
 67: 'see',
 68: 'may',
 69: 'than',
 70: 'world',
 71: 'i',
 72: 'b',
 73: 'would',
 74: 'd',
 75: 'no',
 76: 'however',
 77: 'between',
 78: 'about',
 79: 'over',
 80: 'year

In [8]:
int_words

[5233,
 3080,
 11,
 5,
 194,
 1,
 3133,
 45,
 58,
 155,
 127,
 741,
 476,
 10571,
 133,
 0,
 27349,
 1,
 0,
 102,
 854,
 2,
 0,
 15067,
 58112,
 1,
 0,
 150,
 854,
 3580,
 0,
 194,
 10,
 190,
 58,
 4,
 5,
 10712,
 214,
 6,
 1324,
 104,
 454,
 19,
 58,
 2731,
 362,
 6,
 3672,
 0,
 708,
 1,
 371,
 26,
 40,
 36,
 53,
 539,
 97,
 11,
 5,
 1423,
 2757,
 18,
 567,
 686,
 7088,
 0,
 247,
 5233,
 10,
 1052,
 27,
 0,
 320,
 248,
 44611,
 2877,
 792,
 186,
 5233,
 11,
 5,
 200,
 602,
 10,
 0,
 1134,
 19,
 2621,
 25,
 8983,
 2,
 279,
 31,
 4147,
 141,
 59,
 25,
 6437,
 4186,
 1,
 153,
 32,
 362,
 5233,
 36,
 1137,
 6,
 447,
 344,
 1818,
 19,
 4860,
 0,
 6753,
 1,
 7573,
 1774,
 566,
 0,
 93,
 0,
 247,
 11064,
 11,
 51,
 7088,
 89,
 26,
 270,
 37,
 5948,
 4861,
 20299,
 28,
 53721,
 41,
 317,
 5,
 25636,
 527,
 7573,
 371,
 4,
 258,
 1,
 153,
 25,
 1206,
 11,
 7573,
 200,
 1576,
 2,
 15200,
 332,
 1774,
 7088,
 4860,
 344,
 764,
 160,
 406,
 5690,
 755,
 1,
 4105,
 1131,
 4331,
 1536,
 2,
 567,
 8

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [16]:
## Your code here
import random
# make our frequency associative list
histogram = utils.Counter(int_words)

threshold = 0.01
total_words = len(int_words)
frequencies = { word: count/total_words for word, count in histogram.items() }
word_occurence_probs = { word: 1 - np.sqrt(threshold / frequencies[word]) for word in histogram }

train_words = [word for word in int_words if word_occurence_probs[word] < random.random()] # The final subsampled word list

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [19]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    _range = range(1, window_size)
    r = random.choice(_range)
    lower_bound = idx-r if idx-r >= 0 else 0
    return list(set(words[lower_bound: idx+r+1]))

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [20]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [22]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [24]:
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform([n_vocab, n_embedding], -1, 1)) # create embedding weight matrix here
    embed = tf.nn.embedding_lookup(embedding, inputs)# use tf.nn.embedding_lookup to get the hidden layer output

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [37]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1)) # create softmax weight matrix here
    softmax_b = tf.Variable(tf.zeros(n_vocab)) # create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [38]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [39]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [40]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 7.6850 0.2462 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 7.2188 0.2606 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 6.7029 0.2676 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 6.1669 0.2683 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 5.7472 0.2667 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 5.5525 0.2765 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 5.3181 0.2729 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 5.1898 0.2791 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 4.9762 0.2545 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 4.8767 0.2447 sec/batch
Nearest to war: forbid, tone, never, positive, greatest, opponent, colloquially, kai,
Nearest to five: france, age, chessboard, point, nd, american, hepatic, intent,
Nearest to if: can, aegean, attempting, administrative, parties, exhausting, did, catholic,
Nearest to they: annoyance, frequently, m

Epoch 1/10 Iteration: 4100 Avg. Training loss: 4.4173 0.2347 sec/batch
Epoch 1/10 Iteration: 4200 Avg. Training loss: 4.4146 0.2310 sec/batch
Epoch 1/10 Iteration: 4300 Avg. Training loss: 4.4046 0.2308 sec/batch
Epoch 1/10 Iteration: 4400 Avg. Training loss: 4.4216 0.2307 sec/batch
Epoch 1/10 Iteration: 4500 Avg. Training loss: 4.3884 0.2301 sec/batch
Epoch 1/10 Iteration: 4600 Avg. Training loss: 4.3891 0.2311 sec/batch
Epoch 1/10 Iteration: 4700 Avg. Training loss: 4.4278 0.2322 sec/batch
Epoch 1/10 Iteration: 4800 Avg. Training loss: 4.3918 0.2337 sec/batch
Epoch 1/10 Iteration: 4900 Avg. Training loss: 4.3881 0.2325 sec/batch
Epoch 1/10 Iteration: 5000 Avg. Training loss: 4.3791 0.2318 sec/batch
Nearest to war: kai, forbid, colloquially, darts, graf, never, opponent, greatest,
Nearest to five: bl, superceded, chessboard, age, ackroyd, memorably, juan, nd,
Nearest to if: can, aegean, fraternity, indecent, nomar, sistine, distorted, crossing,
Nearest to they: canadians, annoyance, b

Epoch 1/10 Iteration: 8100 Avg. Training loss: 4.3253 0.2374 sec/batch
Epoch 1/10 Iteration: 8200 Avg. Training loss: 4.2812 0.2349 sec/batch
Epoch 1/10 Iteration: 8300 Avg. Training loss: 4.2767 0.2345 sec/batch
Epoch 1/10 Iteration: 8400 Avg. Training loss: 4.2533 0.2312 sec/batch
Epoch 1/10 Iteration: 8500 Avg. Training loss: 4.2516 0.2294 sec/batch
Epoch 1/10 Iteration: 8600 Avg. Training loss: 4.1892 0.2307 sec/batch
Epoch 1/10 Iteration: 8700 Avg. Training loss: 4.2629 0.2304 sec/batch
Epoch 1/10 Iteration: 8800 Avg. Training loss: 4.2681 0.2318 sec/batch
Epoch 1/10 Iteration: 8900 Avg. Training loss: 4.2844 0.2323 sec/batch
Epoch 1/10 Iteration: 9000 Avg. Training loss: 4.2874 0.2306 sec/batch
Nearest to war: ii, world, darts, kai, contributory, never, liang, forbid,
Nearest to five: bl, superceded, pipelines, ackroyd, acct, age, salazar, pn,
Nearest to if: then, acceleration, wouldn, qt, indecent, flipped, can, nomar,
Nearest to they: their, bakery, were, viejo, saturated, cana

Epoch 1/10 Iteration: 12100 Avg. Training loss: 4.2008 0.2334 sec/batch
Epoch 1/10 Iteration: 12200 Avg. Training loss: 4.2760 0.2324 sec/batch
Epoch 1/10 Iteration: 12300 Avg. Training loss: 4.2683 0.2319 sec/batch
Epoch 1/10 Iteration: 12400 Avg. Training loss: 4.2733 0.2333 sec/batch
Epoch 1/10 Iteration: 12500 Avg. Training loss: 4.2350 0.2327 sec/batch
Epoch 1/10 Iteration: 12600 Avg. Training loss: 4.2128 0.2308 sec/batch
Epoch 1/10 Iteration: 12700 Avg. Training loss: 4.1030 0.2317 sec/batch
Epoch 1/10 Iteration: 12800 Avg. Training loss: 4.1705 0.2306 sec/batch
Epoch 1/10 Iteration: 12900 Avg. Training loss: 4.2220 0.2317 sec/batch
Epoch 1/10 Iteration: 13000 Avg. Training loss: 4.2237 0.2309 sec/batch
Nearest to war: ii, world, cold, civil, contributory, darts, stijl, demanding,
Nearest to five: bl, acct, pipelines, ackroyd, concordance, age, superceded, ini,
Nearest to if: then, wouldn, acceleration, suppose, flipped, indecent, qt, boadicea,
Nearest to they: were, their, excl

Epoch 2/10 Iteration: 16100 Avg. Training loss: 4.0915 0.2326 sec/batch
Epoch 2/10 Iteration: 16200 Avg. Training loss: 4.1716 0.2322 sec/batch
Epoch 2/10 Iteration: 16300 Avg. Training loss: 4.1548 0.2311 sec/batch
Epoch 2/10 Iteration: 16400 Avg. Training loss: 4.1456 0.2313 sec/batch
Epoch 2/10 Iteration: 16500 Avg. Training loss: 4.1289 0.2305 sec/batch
Epoch 2/10 Iteration: 16600 Avg. Training loss: 4.1554 0.2309 sec/batch
Epoch 2/10 Iteration: 16700 Avg. Training loss: 4.1456 0.2321 sec/batch
Epoch 2/10 Iteration: 16800 Avg. Training loss: 4.1697 0.2317 sec/batch
Epoch 2/10 Iteration: 16900 Avg. Training loss: 4.1661 0.2332 sec/batch
Epoch 2/10 Iteration: 17000 Avg. Training loss: 4.1563 0.2332 sec/batch
Nearest to war: world, ii, cold, civil, stijl, conqueror, hrothgar, darts,
Nearest to five: bl, acct, ackroyd, ini, concordance, salazar, smithson, granted,
Nearest to if: acceleration, then, alert, wouldn, harassed, boadicea, crossing, exhausting,
Nearest to they: were, their, d

...

Epoch 3/10 Iteration: 44100 Avg. Training loss: 4.0567 0.2345 sec/batch
Epoch 3/10 Iteration: 44200 Avg. Training loss: 3.9314 0.2296 sec/batch
Epoch 3/10 Iteration: 44300 Avg. Training loss: 4.0519 0.2320 sec/batch
Epoch 3/10 Iteration: 44400 Avg. Training loss: 4.0973 0.2324 sec/batch
Epoch 3/10 Iteration: 44500 Avg. Training loss: 4.0710 0.2304 sec/batch
Epoch 3/10 Iteration: 44600 Avg. Training loss: 4.0157 0.2316 sec/batch
Epoch 3/10 Iteration: 44700 Avg. Training loss: 4.0288 0.2306 sec/batch
Epoch 3/10 Iteration: 44800 Avg. Training loss: 4.0422 0.2298 sec/batch
Epoch 3/10 Iteration: 44900 Avg. Training loss: 4.0365 0.2319 sec/batch
Epoch 3/10 Iteration: 45000 Avg. Training loss: 4.1176 0.2329 sec/batch
Nearest to war: world, ii, cold, civil, peloponnesian, vietnam, russo, second,
Nearest to five: bl, rietveld, hab, smithson, ackroyd, printable, farnborough, acct,
Nearest to if: even, crossing, permitting, harassed, boadicea, fuck, wouldn, then,
Nearest to they: were, their, the

Epoch 4/10 Iteration: 48100 Avg. Training loss: 4.0246 0.2332 sec/batch
Epoch 4/10 Iteration: 48200 Avg. Training loss: 4.1114 0.2302 sec/batch
Epoch 4/10 Iteration: 48300 Avg. Training loss: 4.0724 0.2310 sec/batch
Epoch 4/10 Iteration: 48400 Avg. Training loss: 4.0809 0.2296 sec/batch
Epoch 4/10 Iteration: 48500 Avg. Training loss: 4.0208 0.2305 sec/batch
Epoch 4/10 Iteration: 48600 Avg. Training loss: 4.0526 0.2304 sec/batch
Epoch 4/10 Iteration: 48700 Avg. Training loss: 4.0070 0.2308 sec/batch
Epoch 4/10 Iteration: 48800 Avg. Training loss: 3.8812 0.2285 sec/batch
Epoch 4/10 Iteration: 48900 Avg. Training loss: 4.0210 0.2307 sec/batch
Epoch 4/10 Iteration: 49000 Avg. Training loss: 4.0577 0.2300 sec/batch
Nearest to war: world, ii, cold, civil, peloponnesian, second, post, during,
Nearest to five: zero, rietveld, farnborough, acct, years, hab, masturbated, butter,
Nearest to if: even, then, already, permitting, boadicea, fuck, caracal, revenues,
Nearest to they: their, were, are, 

Epoch 4/10 Iteration: 52100 Avg. Training loss: 4.0589 0.2344 sec/batch
Epoch 4/10 Iteration: 52200 Avg. Training loss: 4.0437 0.2299 sec/batch
Epoch 4/10 Iteration: 52300 Avg. Training loss: 4.0375 0.2290 sec/batch
Epoch 4/10 Iteration: 52400 Avg. Training loss: 4.0791 0.2309 sec/batch
Epoch 4/10 Iteration: 52500 Avg. Training loss: 4.0324 0.2295 sec/batch
Epoch 4/10 Iteration: 52600 Avg. Training loss: 3.9983 0.2307 sec/batch
Epoch 4/10 Iteration: 52700 Avg. Training loss: 4.0520 0.2294 sec/batch
Epoch 4/10 Iteration: 52800 Avg. Training loss: 3.9637 0.2287 sec/batch
Epoch 4/10 Iteration: 52900 Avg. Training loss: 4.0245 0.2294 sec/batch
Epoch 4/10 Iteration: 53000 Avg. Training loss: 4.0706 0.2283 sec/batch
Nearest to war: world, ii, cold, civil, post, peloponnesian, russo, vietnam,
Nearest to five: acct, rietveld, untold, farnborough, printable, hab, butter, ackroyd,
Nearest to if: even, already, permitting, crossing, then, acceleration, revenues, caracal,
Nearest to they: their, a

Epoch 4/10 Iteration: 56100 Avg. Training loss: 4.0477 0.2323 sec/batch
Epoch 4/10 Iteration: 56200 Avg. Training loss: 3.9901 0.2305 sec/batch
Epoch 4/10 Iteration: 56300 Avg. Training loss: 4.1035 0.2327 sec/batch
Epoch 4/10 Iteration: 56400 Avg. Training loss: 3.8490 0.2274 sec/batch
Epoch 4/10 Iteration: 56500 Avg. Training loss: 3.7068 0.2278 sec/batch
Epoch 4/10 Iteration: 56600 Avg. Training loss: 3.9643 0.2274 sec/batch
Epoch 4/10 Iteration: 56700 Avg. Training loss: 3.8477 0.2280 sec/batch
Epoch 4/10 Iteration: 56800 Avg. Training loss: 3.9408 0.2301 sec/batch
Epoch 4/10 Iteration: 56900 Avg. Training loss: 4.0229 0.2299 sec/batch
Epoch 4/10 Iteration: 57000 Avg. Training loss: 4.0416 0.2317 sec/batch
Nearest to war: world, ii, cold, civil, peloponnesian, second, vietnam, russo,
Nearest to five: printable, bl, walser, schrader, rietveld, hab, farnborough, smithson,
Nearest to if: even, already, permitting, fuck, then, boadicea, you, wouldn,
Nearest to they: their, were, are, t

KeyboardInterrupt: 

Restore the trained network if you need to:

In [41]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

INFO:tensorflow:Restoring parameters from None


SystemError: <built-in function TF_Run> returned a result with an error set

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)